- SAD
- MSE
- structural similarity
- F1


In [1]:
from skimage import metrics
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.metrics import adapted_rand_error, variation_of_information
from sklearn.metrics import confusion_matrix  

In [2]:
def sad(imageA, imageB):
   
    # sum of the abs difference between the two images;
    err = np.sum(np.abs(imageA.astype("float") - imageB.astype("float")))
    err /= float(imageA.shape[0] * imageA.shape[1])

    return err

In [3]:
def mse(imageA, imageB):
    
    # sum of the squared difference between the two images;
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    return err


In [4]:
def compute_iou(y_pred, y_true):
    # ytrue, ypred is a flatten vector
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    current = confusion_matrix(y_true, y_pred, labels=[0, 1])
    # compute mean iou
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    return np.mean(IoU)

In [5]:
def compare_images(imageA, imageB, title):

    m = mse(imageA, imageB)
    s = metrics.structural_similarity(imageA, imageB)
    d = sad(imageA, imageB)
    
    error, precision, recall = adapted_rand_error(imageA, imageB)
    splits, merges = variation_of_information(imageA, imageB)

    f1 = 2 * precision * recall / (precision + recall)
    iou = compute_iou(imageA, imageB)

#     fig = plt.figure(title)
#     plt.suptitle("MSE: %.2f, SAD: %.2f, SSIM: %.2f, F1: %.2f, IoU: %.2f" % (m, d, s, f1, iou))

#     ax = fig.add_subplot(1, 2, 1)
#     plt.imshow(imageA, cmap = plt.cm.gray)
#     plt.axis("off")

#     ax = fig.add_subplot(1, 2, 2)
#     plt.imshow(imageB, cmap = plt.cm.gray)
#     plt.axis("off")

#     plt.show()

#     print('mse: ', m)
#     print('sad: ', d)
#     print('structural similarity: ', s)

#     print('f1: ', f1)
#     print('iou: ', iou)
    
#     print('precision: ', precision)
#     print('recall: ', recall)

#     print('mse, sad, ssim, f1')

    return m, d, s, f1

In [6]:
# impath1 = 'test/test.png'
# impath2 = 'test/test_binarized.png'

impath1 = 'objective_test/original_gts/00.png'
impath2 = 'objective_test/our_masks/00.png'
impath3 = 'objective_test/removebg_masks/00.png'

In [7]:
img1 = cv2.imread(impath1)
img2 = cv2.imread(impath2)
img3 = cv2.imread(impath3)

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

In [8]:
compare_images(img1, img3, "Original vs. Removebg")

(382.53786849975586,
 2.4685401916503906,
 0.9648392165800469,
 0.9775524171619054)

In [9]:
compare_images(img1, img2, "Original vs. Ours")

(93.5726547241211, 1.3487930297851562, 0.9803990708630027, 0.9238380091223866)

In [10]:
compare_images(img3, img2, "Removebg vs. Ours")

(230.21543502807617,
 2.0574684143066406,
 0.9700862875488642,
 0.8960130577704767)

In [11]:
import glob

In [12]:
path1_gt = 'objective_test/original_gts/'
path2_nano = 'objective_test/our_masks/'
path3_rmbg = 'objective_test/removebg_masks/'

In [13]:
gt_files = glob.glob(path1_gt+'*')
nano_files = glob.glob(path2_nano+'*')
rmbg_files = glob.glob(path3_rmbg+'*')

In [14]:
gt_files.sort()
nano_files.sort()
rmbg_files.sort()

In [15]:
len(gt_files), len(nano_files), len(rmbg_files)

(1922, 1922, 1922)

In [16]:
gt_files[1].split('/')[-1]

'00001.png'

In [17]:
gt_files[437].split('/')[-1]

'302432h09_38.png'

In [18]:
nano_files[437].split('/')[-1]

'302432h09_38.png'

In [19]:
rmbg_files[437].split('/')[-1]

'302432h09_38.png'

In [20]:
#check 

for i in range (len(gt_files)):
    if not((gt_files[i].split('/')[-1] == nano_files[i].split('/')[-1]) and (nano_files[i].split('/')[-1]==rmbg_files[i].split('/')[-1])):
        print(i)

In [21]:
import time

In [ ]:
mse_nano = []
mse_rmbg = []

sad_nano = []
sad_rmbg = []

ssim_nano = []
ssim_rmbg = []

f1_nano = []
f1_rmbg = []

start = time.monotonic()
for i in range(len(gt_files)):
    
    img1 = cv2.imread(gt_files[i]) # эталон
    img2 = cv2.imread(nano_files[i]) # наша
    img3 = cv2.imread(rmbg_files[i]) # их 

    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
    
    try:
        res1 = compare_images(img1, img2, "Original vs. Ours")
        res2 = compare_images(img1, img3, "Original vs. Removebg")

        mse_nano.append(res1[0])
        mse_rmbg.append(res2[0])

        sad_nano.append(res1[1])
        sad_rmbg.append(res2[1])

        ssim_nano.append(res1[2])
        ssim_rmbg.append(res2[2])

        f1_nano.append(res1[3])
        f1_rmbg.append(res2[3])
    except:
        print(i)

end = time.monotonic() - start

In [23]:
end

787.4070181490388

In [24]:
print('Меньше - лучше: ')
print(' ')
print('MSE nanosemanitcs: ', sum(mse_nano))
print('MSE remove.bg : ', sum(mse_rmbg))
print(' ')

print('SAD nanosemanitcs: ', sum(sad_nano))
print('SAD remove.bg : ', sum(sad_rmbg))
print(' ')
print(' ')
print(' ')

print('Больше - лучше: ')
print(' ')

print('SSim nanosemanitcs: ', sum(ssim_nano))
print('SSim remove.bg : ', sum(ssim_rmbg))
print(' ')

print('F1 nanosemanitcs: ', sum(f1_nano))
print('F1 remove.bg : ', sum(f1_rmbg))

Меньше - лучше: 
 
MSE nanosemanitcs:  278382.10261917114
MSE remove.bg :  1373062.7342147827
 
SAD nanosemanitcs:  4143.231281280518
SAD remove.bg :  7426.797630310059
 
 
 
Больше - лучше: 
 
SSim nanosemanitcs:  1817.6073532942585
SSim remove.bg :  1743.951858394521
 
F1 nanosemanitcs:  1646.8145118626246
F1 remove.bg :  1880.0310921851362
